In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
# mount Google Drive
from os.path import expanduser


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('iphonevaluemoney.csv')
df.head(5)

,stars,comment
0,5,If you are an Android user who wants to move t...
1,5,After 3 years with iPhone 7 I finally decided ...
2,5,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...
3,4,"Battery Life-Can't Complain much on this, pret..."
4,5,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    1100 non-null   int64 
 1   comment  1100 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.3+ KB


In [5]:
df.describe().round(1)

,stars
count,1100.0
mean,4.4
std,0.7
min,3.0
25%,4.0
50%,4.5
75%,5.0
max,5.0


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [7]:
df= df.dropna()
df.shape

(1100, 2)

In [8]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,If you are an Android user who wants to move t...,50
1,After 3 years with iPhone 7 I finally decided ...,30
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,17
3,"Battery Life-Can't Complain much on this, pret...",71
4,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...,56


In [10]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [11]:
df[['comment','punctuation']].head()

,comment,punctuation
0,If you are an Android user who wants to move t...,14
1,After 3 years with iPhone 7 I finally decided ...,11
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,9
3,"Battery Life-Can't Complain much on this, pret...",27
4,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...,7


In [12]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,If you are an Android user who wants to move t...,0
1,After 3 years with iPhone 7 I finally decided ...,0
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0
3,"Battery Life-Can't Complain much on this, pret...",0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

0

In [14]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,If you are an Android user who wants to move t...,1
1,After 3 years with iPhone 7 I finally decided ...,3
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0
3,"Battery Life-Can't Complain much on this, pret...",2
4,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...,0


In [15]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,If you are an Android user who wants to move t...,4
1,After 3 years with iPhone 7 I finally decided ...,2
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0
3,"Battery Life-Can't Complain much on this, pret...",4
4,\n\n\n\n\n\n\n\n\n\n \n \n \n The apple ...,4


In [16]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    if you are an android user who wants to move t...
1    after 3 years with iphone 7 i finally decided ...
2    awesome phone!! specially if you’re upgrading ...
3    battery life-can't complain much on this, pret...
4    the apple iphone is always best in class when ...
Name: comment, dtype: object

In [17]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-17-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    if you are an android user who wants to move t...
1    after 3 years with iphone 7 i finally decided ...
2    awesome phone specially if youre upgrading fro...
3    battery lifecant complain much on this pretty ...
4    the apple iphone is always best in class when ...
Name: comment, dtype: object

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

407    read lot negative reviews regarding buying iph...
203    battery lifecant complain much pretty decent 5...
311    3 years iphone 7 finally decided take upgrade ...
380    android user wants move ios best iphone get wi...
880    android user wants move ios best iphone get wi...
360    android user wants move ios best iphone get wi...
209    first sceptical purchase shifting android ios ...
75     great iphone looking long term usage durabilit...
879    first sceptical purchase shifting android ios ...
425    great iphone looking long term usage durabilit...
Name: comment, dtype: object

In [19]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [20]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [21]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [22]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [23]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [24]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [25]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [26]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [28]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [29]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [30]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    andros user wants move is best phone get witho...
1    3 years phone 7 finally decided take upgrade p...
2    awesome phone specially your upbraiding older ...
3    battery lifecant complain much pretty decent 5...
4    apple phone always best class purchased dismou...
Name: comment, dtype: object

In [31]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [32]:
df['comment'] = df.comment.apply(round1)
df.comment

0       android user wants move ios best iphone get wi...
1        years iphone  finally decided take upgrade ip...
2       awesome phone specially youre upgrading older ...
3       battery lifecant complain much pretty decent  ...
4       apple iphone always best class purchased disco...
                              ...                        
1095    great iphone looking long term usage durabilit...
1096    honest feel like paid price brand iphone defin...
1097    read lot negative reviews regarding buying iph...
1098    u read review know many bad comments also intr...
1099    first sceptical purchase shifting android ios ...
Name: comment, Length: 1100, dtype: object

In [33]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [34]:
df['comment'] = df.comment.apply(round2)
df.comment

0       android user wants move ios best iphone get wi...
1        years iphone  finally decided take upgrade ip...
2       awesome phone specially youre upgrading older ...
3       battery lifecant complain much pretty decent  ...
4       apple iphone always best class purchased disco...
                              ...                        
1095    great iphone looking long term usage durabilit...
1096    honest feel like paid price brand iphone defin...
1097    read lot negative reviews regarding buying iph...
1098    u read review know many bad comments also intr...
1099    first sceptical purchase shifting android ios ...
Name: comment, Length: 1100, dtype: object

In [35]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

iphone     1980
android    1650
phone      1430
apple      1210
value      1210
money      1210
also        990
ios         990
good        880
use         880
first       770
battery     770
buy         770
get         770
great       770
using       770
best        770
time        660
pro         660
might       660
dtype: int64

In [36]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,android user wants move ios best iphone get wi...,53
1,years iphone finally decided take upgrade ip...,47
2,awesome phone specially youre upgrading older ...,28
3,battery lifecant complain much pretty decent ...,81
4,apple iphone always best class purchased disco...,75


In [37]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [38]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,android user wants move ios best iphone get wi...,339
1,years iphone finally decided take upgrade ip...,276
2,awesome phone specially youre upgrading older ...,200
3,battery lifecant complain much pretty decent ...,541
4,apple iphone always best class purchased disco...,532


In [39]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [40]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,android user wants move ios best iphone get wi...,5.6
1,years iphone finally decided take upgrade ip...,5.6
2,awesome phone specially youre upgrading older ...,6.2
3,battery lifecant complain much pretty decent ...,5.9
4,apple iphone always best class purchased disco...,6.1


In [41]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
790,5,android user wants move ios best iphone get wi...,50,14,0,1,4,53,339,5.6
262,5,awesome phone specially youre upgrading older ...,17,9,0,0,0,28,200,6.2


In [42]:
#df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [43]:
#df= df.drop('ProfileName', axis= 1)

In [44]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [45]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
807,4,read lot negative reviews regarding buying iph...,75,9,0,1,3,89,575,5.7
995,4,great iphone looking long term usage durabilit...,30,3,0,1,0,41,274,5.8
342,5,awesome phone specially youre upgrading older ...,17,9,0,0,0,28,200,6.2
885,5,apple iphone always best class purchased disco...,56,7,0,0,4,75,532,6.1
1025,4,great iphone looking long term usage durabilit...,30,3,0,1,0,41,274,5.8


In [46]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes"      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [47]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

780     android user wants move ios best iphone withou...
965     great iphone looking long term usage durabilit...
486     honest feel paid price brand iphone definitely...
34      apple iphone always best class purchased disco...
1074    apple iphone always best class purchased disco...
177     read lot negative reviews regarding buying iph...
297     read lot negative reviews regarding buying iph...
234     apple iphone always best class purchased disco...
53      battery lifecant complain much pretty decent w...
661     years iphone finally decided take upgrade ipho...
Name: comment, dtype: object

In [48]:
df1= df

In [49]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [50]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [51]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0       android user wants move ios best iphone withou...
1       years iphone finally decided take upgrade ipho...
2       awesome phone specially youre upgrading older ...
3       battery lifecant complain much pretty decent w...
4       apple iphone always best class purchased disco...
                              ...                        
1095    great iphone looking long term usage durabilit...
1096    honest feel paid price brand iphone definitely...
1097    read lot negative reviews regarding buying iph...
1098    read review know many bad comments also intrig...
1099    sceptical purchase shifting android ios line s...
Name: comment, Length: 1100, dtype: object

In [52]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [53]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [54]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [55]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

iphone        1980
android       1650
phone         1430
value         1210
money         1210
apple         1210
also           990
ios            990
good           880
best           770
battery        770
buy            770
great          770
using          770
pro            660
amazon         660
time           660
dont           660
might          660
price          550
long           550
small          550
charger        550
want           550
easy           550
life           550
experience     550
lot            550
always         550
takes          550
one            440
brand          440
way            440
camera         440
away           440
would          440
quality        440
usage          440
wont           440
many           440
even           440
display        440
bill           440
makes          440
oneplus        330
features       330
heavy          330
heating        330
go             330
cover          330
dtype: int64

In [56]:
df.to_csv('iphonevaluemoney2.csv', index=False)